# Summary

- What it does
    - Recipe management/storage
        - Input recipe, output df, csv, txt
    - Recipe costing
    - ?Restaurant simulation
        - Point rewards
        - Blockchain payments
- Who it helps
- What tools/libraries does it use

## To Do:
- Set up Spoonacular API
    - Explore to relevant endpoints
    - Create functions to get data from endpoints


## Resources
[Python Dev Interface: Requests](https://docs.python-requests.org/en/latest/api/)

# Imports, Base Functions

In [5]:
from pathlib import Path
import csv
import pandas as pd
import panel as pn
pn.extension('plotly')
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe_connected'  # Bypass mimetype 'renderer not found'
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
# import numpy as np
# import seaborn as sns

import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv('token.env')
spoon_key = os.getenv('SPOONACULAR_KEY')
gcloud_key = os.getenv('GCLOUD_KEY')
gsheets_oauth = os.getenv('GSHEETS_OAUTH')

import json, requests
from pandas.io.json import json_normalize

# import alpaca_trade_api as tradeapi

# Import `chatter` functions from chatter.py
import sys
from chatter import *

# Spoonacular
API with endpoints related to food, including ingredient price and nutrition data, recipe cost breakdown, product comparisons, and menu data from over 800 American restaurant chains.

- 

## API and key URLs

In [3]:
# Requests should be formatted as "api + endpoint + "?query=" + {query_params} + api_key
spoon_url = 'https://api.spoonacular.com'
key_url = '&apiKey=' + spoon_key
# requests.get(spoon_url + key_url + spoon_key)

## Endpoints
Learn more about [quotas](https://spoonacular.com/food-api/docs#Quotas).

### [Search Recipes](https://spoonacular.com/food-api/docs#Search-Recipes-Complex)

Search through hundreds of thousands of recipes using advanced filtering and ranking.

NOTE: This method combines searching by query, by ingredients, and by nutrients into one endpoint.

Calling this endpoint requires `1 point` and `0.01 points` per result returned. Since this endpoint combines the capabilities of four different endpoints into one, additional points may be required depending on the parameters you set.

- If `fillIngredients` is set to true, `0.025 points` will be added per recipe returned.
- If a nutrient filter is set, `1 point` will be added.
- If `addRecipeInformation` is set to true, `0.025 points` will be added per recipe returned.
- If `addRecipeNutrition` is set to true, `0.025 points` will be added per recipe returned and `addRecipeInformation` will automatically be set to true as well.

Parameters:
- `query`: the (natural language) recipe search query.
- `cuisine`: the cuisine(s) of the recipes. One or more, comma separated. [List of supported cuisines](https://spoonacular.com/food-api/docs#Cuisines)
- `excludeCuisine`: the cuisine(s) the recipes must not match. One or more, comma separated.
- `includeIngredients`: a comma-separated list of ingredients that should/must be used in the recipes.
- `excludeIngredients`: a comma-separated list of ingredients or ingredient types that the recipes must not contain.

In [21]:
# Search Recipes endpoint
search_recipe_url = spoon_url + '/recipes/complexSearch?query='

# Function that takes in a query, ingredient, or nutrient and returns up to 10 relevant recipes
# Example request: https://api.spoonacular.com/recipes/complexSearch?query=pasta&maxFat=25&number=2
def search_recipe(query):
    return requests.get(search_recipe_url + query + '&number=10' + key_url).json()

search_recipe('cookies')

{'results': [{'id': 654028,
   'title': 'Oreo Cookies & Cream No-Bake Cheesecake',
   'image': 'https://spoonacular.com/recipeImages/654028-312x231.jpg',
   'imageType': 'jpg'},
  {'id': 662151,
   'title': 'Sugar Cookies',
   'image': 'https://spoonacular.com/recipeImages/662151-312x231.jpg',
   'imageType': 'jpg'},
  {'id': 1095878,
   'title': 'Muesli Cookies',
   'image': 'https://spoonacular.com/recipeImages/1095878-312x231.jpg',
   'imageType': 'jpg'},
  {'id': 639865,
   'title': 'Coffee Cookies',
   'image': 'https://spoonacular.com/recipeImages/639865-312x231.jpg',
   'imageType': 'jpg'},
  {'id': 662786,
   'title': 'Tahini Cookies',
   'image': 'https://spoonacular.com/recipeImages/662786-312x231.jpg',
   'imageType': 'jpg'},
  {'id': 655353,
   'title': 'Peanut Cookies',
   'image': 'https://spoonacular.com/recipeImages/655353-312x231.jpg',
   'imageType': 'jpg'},
  {'id': 640246,
   'title': 'Cowboy Cookies With Pretzels and Raisinettes',
   'image': 'https://spoonacular.c

### [Search Recipes by Ingredients](https://spoonacular.com/food-api/docs#Search-Recipes-by-Ingredients)

Ever wondered what recipes you can cook with the ingredients you have in your fridge or pantry? This endpoint lets you find recipes that either maximize the usage of ingredients you have at hand (pre shopping) or minimize the ingredients that you don't currently have (post shopping).

Find recipes that use as many of the given ingredients as possible and require as few additional ingredients as possible. This is a "what's in your fridge" API endpoint.

Calling this endpoint requires `1 point` and `0.01 points` per recipe returned.

Parameters:
- `ingredients`: a comma-separated list of ingredients that the recipes should contain
- `number` the maximum number of recipes to return (between 1 and 100). Default 10.

In [22]:
recipe_by_ing_url = spoon_url + '/recipes/findByIngredients?ingredients='

def search_recipe_by_ing():
    pass

### [Ingredient Search](https://spoonacular.com/food-api/docs#Ingredient-Search)

Search for simple whole foods (e.g. fruits, vegetables, nuts, grains, meat, fish, dairy etc.).

Calling this endpoint requires `1 point` and `0.01 points` per result if `metaInformation` is set to true.

Parameters:
- `query`: the partial or full ingredient name.
- `number`: the number of expected results (between 1 and 100).

In [23]:
# Ingredient Search endpoint
search_ing_url = spoon_url + '/food/ingredients/search?query='

# Function that takes in an ingredient and returns up to 20 matches
# Example request: https://api.spoonacular.com/food/ingredients/search?query=banana&number=2&sort=calories&sortDirection=desc
def search_ing(ing):
    return requests.get(search_ing_url + ing + '&number=20' + key_url).json()

search_ing('chocolate')

{'results': [{'id': 19081, 'name': 'chocolate', 'image': 'milk-chocolate.jpg'},
  {'id': 1102, 'name': 'chocolate milk', 'image': 'chocolate-milk.jpg'},
  {'id': 1002064, 'name': 'chocolate mint', 'image': 'mint.jpg'},
  {'id': 99278, 'name': 'chocolate chips', 'image': 'chocolate-chips.jpg'},
  {'id': 14181, 'name': 'chocolate syrup', 'image': 'chocolate-syrup.jpg'},
  {'id': 10119903, 'name': 'chocolate curls', 'image': 'chocolate-curls.jpg'},
  {'id': 10019348, 'name': 'chocolate sauce', 'image': 'chocolate-glaze.png'},
  {'id': 10419903, 'name': 'chocolate chunks', 'image': 'chocolate-chips.jpg'},
  {'id': 10118157,
   'name': 'chocolate wafers',
   'image': 'chocolate-wafer-cookies.jpg'},
  {'id': 19375, 'name': 'chocolate glaze', 'image': 'chocolate-glaze.png'},
  {'id': 19226,
   'name': 'chocolate frosting',
   'image': 'chocolate-frosting.png'},
  {'id': 18157,
   'name': 'chocolate cookies',
   'image': 'chocolate-wafer-cookies.jpg'},
  {'id': 10219903,
   'name': 'chocolate 

### [Get Ingredient Information](https://spoonacular.com/food-api/docs#Get-Ingredient-Information)
Use an ingredient id to get all available information about an ingredient, such as its image and supermarket aisle.

Calling this endpoint requires `1 point`.

Parameters:
- `id`: the ingredient ID#
- `amount`: the amount of this ingredient
- `unit`: the unit of measure for the given amount (e.g. grams)

In [85]:
# Get ingredient information
def get_ing_info(ing_id, ing_amount, unit):
        return requests.get(f"{spoon_url}/food/ingredients/{ing_id}/information?amount={ing_amount}&unit={unit}{key_url}").json()

In [86]:
# Create df with desired ingredient info
def ing_info_to_df(ing_id, ing_amount, unit):
    ing_info = get_ing_info(ing_id, ing_amount, unit)
    return pd.json_normalize(ing_info)[['id','name','amount','unit','possibleUnits','estimatedCost.value','estimatedCost.unit']]

#### Tests

In [87]:
# Test of get_ing_info()
test_chocolate = get_ing_info('99278','200','g')
test_chocolate

{'id': 99278,
 'original': 'chocolate morsels',
 'originalName': 'chocolate morsels',
 'name': 'chocolate morsels',
 'amount': 200.0,
 'unit': 'g',
 'unitShort': 'g',
 'unitLong': 'grams',
 'possibleUnits': ['g', 'oz'],
 'estimatedCost': {'value': 135.71, 'unit': 'US Cents'},
 'consistency': 'solid',
 'aisle': 'Baking',
 'image': 'chocolate-chips.jpg',
 'meta': [],
 'nutrition': {'nutrients': [{'title': 'Mono Unsaturated Fat',
    'name': 'Mono Unsaturated Fat',
    'amount': 0.0,
    'unit': 'g'},
   {'title': 'Fat', 'name': 'Fat', 'amount': 28.26, 'unit': 'g'},
   {'title': 'Vitamin B3', 'name': 'Vitamin B3', 'amount': 0.0, 'unit': 'mg'},
   {'title': 'Carbohydrates',
    'name': 'Carbohydrates',
    'amount': 63.6,
    'unit': 'g'},
   {'title': 'Vitamin A', 'name': 'Vitamin A', 'amount': 0.0, 'unit': 'IU'},
   {'title': 'Vitamin B6', 'name': 'Vitamin B6', 'amount': 0.0, 'unit': 'mg'},
   {'title': 'Iron', 'name': 'Iron', 'amount': 0.0, 'unit': 'mg'},
   {'title': 'Potassium', 'name

In [92]:
# Extracting desired data from ingredient info
print(f"{test_chocolate['estimatedCost']['value']} {test_chocolate['estimatedCost']['unit']}")
pd.json_normalize(test_chocolate)[['id','name','amount','unit','possibleUnits','estimatedCost.value','estimatedCost.unit']]

135.71 US Cents


,id,name,amount,unit,possibleUnits,estimatedCost.value,estimatedCost.unit
0,99278,chocolate morsels,200.0,g,"[g, oz]",135.71,US Cents


In [91]:
# Test of ing_info_to_df()
ing_info_to_df('99278','1','g')

,id,name,amount,unit,possibleUnits,estimatedCost.value,estimatedCost.unit
0,99278,chocolate morsels,1.0,g,"[g, oz]",0.68,US Cents


### [Convert Amounts](https://spoonacular.com/food-api/docs#Convert-Amounts)
Convert unit measurements like "2.5 cups of flour to grams".

Parameters:
- `ingredientName`: the ingredient to convert.
- `sourceAmount`: the original amount you're converting **<u>from</u>** (e.g. the "2.5" in "2.5 cups of flour to grams").
- `sourceUnit`: the original unit of measure you're converting **<u>from</u>** (e.g. the "cups" in "2.5 cups of flour to grams").
- `targetUnit`: the new unit of measure you're converting **<u>to</u>** (e.g. the "grams" in "2.5 cups of flour to grams").

In [25]:
# Function to convert an ingredient's unit of measure
# Example request: https://api.spoonacular.com/recipes/convert?ingredientName=flour&sourceAmount=2.5&sourceUnit=cups&targetUnit=grams
def convert(ing, sourceAmount, sourceUnit, targetUnit):
    return requests.get(f"{spoon_url}/recipes/convert?ingredientName={ing}&sourceAmount={str(sourceAmount)}&sourceUnit={sourceUnit}&targetUnit={targetUnit}{key_url}").json()

convert('flour', 2.5, 'cups', 'grams')

{'sourceAmount': 2.5,
 'sourceUnit': 'cups',
 'targetAmount': 312.5,
 'targetUnit': 'grams',
 'answer': '2.5 cups flour translates to 312.5 grams.',
 'type': 'CONVERSION'}

In [26]:
# Test convert() function and separating data
test_convert = convert('flour', 2.5, 'cups', 'grams')
print(f"{test_convert['sourceAmount']} {test_convert['sourceUnit']}")
print(f"{test_convert['targetAmount']} {test_convert['targetUnit']}")

2.5 cups
312.5 grams


In [27]:
# Create a DataFrame from the convert() json data
test_df = pd.json_normalize(test_convert).drop(columns=['answer','type'])
test_df

,sourceAmount,sourceUnit,targetAmount,targetUnit
0,2.5,cups,312.5,grams


### Other Endpoints

In [ ]:
# # Ingredients endpoints
# search_ingredient = spoon_url + '/food/ingredients/search?query='

# # Products endpoints
# products = spoon_url + '/food/products'

# # Menu Items endpoints
# menu_item = spoon_url + '/food/menuItems'

# # Meal Planner endpoints
# meal_plan = spoon_url + '/mealplanner'

# # Wine endpoints
# wine = spoon_url + '/food/wine'

# # Detect Food in Text endpoint
# detect = spoon_url + '/food/detect'

In [29]:
# Get ID from search results


In [30]:
# # Dictionary containing endpoints
# ends = {
#     'recipes': '/recipes/complexSearch?',
#     'ingredient': '/food/ingredients?',
#     'products': '/food/products?',
#     'menu_items': '/food/menuItems?',
#     'meal_plan': '/mealplanner?',
#     'wine': '/food/wine?',
#     'detect': '/food/detect?',
# }

# def search(endpoint, query_item):
#     # user_ends = input("Select endpoint: ")
#     search_url = f"{spoon_url}{ends[endpoint]}query={query_item}{key_url}"
#     print(search_url)

In [31]:
# search("recipes", "mango")

https://api.spoonacular.com/recipes/complexSearch?query=mango&apiKey=1dbea58c8d314428b670f768096872bc


In [32]:
# search_ing + 'banana' + api_key + '?format=json'
# requests.get(z)
# y = search_ing + 'mango' + api_key #+ '?format=json'
# print(y)
# requests.get(y)

In [33]:
# z = search_ing + '/search?' + api_key + '&query=mango'
# print(z)
# requests.get(z)

In [36]:
# def find_ing(ing):
#     search_ing + ing + api_key
# find_ing("banana")

In [42]:
request = requests.get(spoon_url, params={
    'grant_type': 'client_credentials',
    'client_id': spoon_key,
})
{request}

{<Response [404]>}

In [ ]:
request = requests.post(spoon_url + '?apiKey=' + spoon_key, {
    'grant_type': 'client_credentials',
    'client_id': api_key,
})

In [ ]:
request

In [ ]:
recipe_id = ''
ingredient = ''
x = ''
class spoon:
    class recipes():
        recipes = api + 'recipes/'
        search = recipes + 'complexSearch/'
        price = recipes + recipe_id + 'priceBreakdownWidget.json'
        ingredients = recipes + recipe_id + 'ingredientWidget.json'
        nutrition = recipes + recipe_id + 'nutritionWidget.json'
    class ingredient(x):
        ingredient = api + 'food/ingredients/'
        search = ingredient + 'search?query=' + x

In [ ]:
# Imports
import time
import sys
from random import randrange

# List for holding 'chatter' text
npc_chatter = []

# Function is based off an example found in Stack Overflow
def chatter(npc_chatter):
    print()
    for c in npc_chatter:
        sys.stdout.write(c)
        sys.stdout.flush()
        secs = '0.0' + str(randrange(10,20,1))
        secs = float(secs)
        time.sleep(secs)
    print()
    time.sleep(2)

# 'Chatter' function test using an appended list of dialog
# Though this aspect of the function is not utilized, it is kept in case it proves useful later
# If the line break (\n) is desired, double tab to align the separated lines
test_chatter = 'NPC: "This is just a test to see if your function functions,\n      and it\'s functioning just fine, friend!"'
npc_chatter.append(test_chatter)
chatter(test_chatter)

# Google Sheets API
[Google Sheets for Developers: Sheets API](https://developers.google.com/sheets/api/reference/rest)

In [ ]:
# Google Client Library Installation
# !#pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

# Imports for Google Cloud
from googleapiclient import discovery
from oauth2client.client import OAuth2Credentials as creds
crm = discovery.build(
    'cloudresourcemanager', 'v3', http=creds.authorize(httplib2.Http()))

project = crm.projects().get(projectId=flags.projectId).execute()

In [ ]:
# from __future__ import print_function
# import os.path
# from googleapiclient.discovery import build
# from google_auth_oauthlib.flow import InstalledAppFlow
# from google.auth.transport.requests import Request
# from google.oauth2.credentials import Credentials

# # If modifying these scopes, delete the file token.json.
# SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# # The ID and range of a sample spreadsheet.
# SAMPLE_SPREADSHEET_ID = '1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms'
# SAMPLE_RANGE_NAME = 'Class Data!A2:E'

# def main():
#     """Shows basic usage of the Sheets API.
#     Prints values from a sample spreadsheet.
#     """
#     creds = None
#     # The file token.json stores the user's access and refresh tokens, and is
#     # created automatically when the authorization flow completes for the first
#     # time.
#     if os.path.exists('token.json'):
#         creds = Credentials.from_authorized_user_file('token.json', SCOPES)
#     # If there are no (valid) credentials available, let the user log in.
#     if not creds or not creds.valid:
#         if creds and creds.expired and creds.refresh_token:
#             creds.refresh(Request())
#         else:
#             flow = InstalledAppFlow.from_client_secrets_file(
#                 'credentials.json', SCOPES)
#             creds = flow.run_local_server(port=0)
#         # Save the credentials for the next run
#         with open('token.json', 'w') as token:
#             token.write(creds.to_json())

#     service = build('sheets', 'v4', credentials=creds)

#     # Call the Sheets API
#     sheet = service.spreadsheets()
#     result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
#                                 range=SAMPLE_RANGE_NAME).execute()
#     values = result.get('values', [])

#     if not values:
#         print('No data found.')
#     else:
#         print('Name, Major:')
#         for row in values:
#             # Print columns A and E, which correspond to indices 0 and 4.
#             print('%s, %s' % (row[0], row[4]))

# if __name__ == '__main__':
#     main()

In [ ]:
# Google Sheets service endpoint
sheets_url = 'https://sheets.googleapis.com'
sheets_response = requests.get(sheets_url)
sheets_response

In [ ]:
# https://developers.google.com/sheets/api/reference/rest/v4/ValueRenderOption

## [GSpread by burnash](https://github.com/burnash/gspread)

In [2]:
# \!pip install gspread

# Product List CSV

# Recipe Cost CSV


# News API
API for searching articles and breaking news headlines from around the world, even in other languages.